In [14]:
!poetry update

Updating dependencies
Resolving dependencies... (9.8s)Resolving dependencies... (2.6s)

No dependencies to install or update


# Option valuation model based on historical prices

In [1]:
import importlib.util
import os
google_colab_spec = importlib.util.find_spec("google") and importlib.util.find_spec("google.colab")
if google_colab_spec:
    google_colab = importlib.import_module('google.colab')
    google_colab.drive.mount('/content/drive', force_remount=True)
    path = '/content/drive/MyDrive/Colab Notebooks/Ethereum Prices'
else:
    path = f'{os.environ["HOME"]}/Library/CloudStorage/GoogleDrive-chris.perso@gmail.com/My Drive/Colab Notebooks/Ethereum Prices'

import optionspricing
import binanceprices

COUNT_YEARS = 10
BINANCE_DATETIME_FORMAT = "%Y-%m-%d %H-%M-%S"
BINANCE_SYMBOL = 'ETHUSDT'

prices_df = binanceprices.load_prices(path, BINANCE_SYMBOL, COUNT_YEARS)



loading 2014 ............
loading 2015 ............
loading 2016 ............
loading 2017 ............
loading 2018 ............
loading 2019 ............
loading 2020 ............
loading 2021 ............
loading 2022 ............
loading 2023 ............
loading 2024 
interrupting at 2024/01


## Option valuation model: input parameters here

In [8]:
target_period_hours = 3 * 24
strikes_universe_size = 4
CUT_OFF_YEAR_MONTH = (2022, 1)

instrument_code = BINANCE_SYMBOL[:3]

headers = {"Content-Type": "application/json"}
base_url = "https://www.deribit.com/api/v2/public"

trading_model = optionspricing.TradingModel(base_url, headers, instrument_code, target_period_hours)
trading_model.cutoff_year_month(CUT_OFF_YEAR_MONTH)
target_expiry = trading_model.target_expiry
underlying_price = trading_model.underlying_price
remaining_hours = trading_model.remaining_hours

simulation = trading_model.evaluate(prices_df, strikes_universe_size)
print(simulation)
simulation.allocate([1, 2], 1500. * 2./100.)

target expiry: 2024-02-01 08:00:00+00:00 (70 hours left)
current price: 2270.17
-------------------------------
trading put 2250 and call 2300 (hit ratio: 47%)
cost: 0.021 / value: 0.034, benefit/cost = 1.6x
($) cost: 47.67 / value: 76.80, average gain = 29.12
-------------------------------
trading put 2225 and call 2325 (hit ratio: 43%)
cost: 0.014 / value: 0.026, benefit/cost = 1.9x
($) cost: 31.78 / value: 59.68, average gain = 27.90
-------------------------------
trading put 2200 and call 2350 (hit ratio: 37%)
cost: 0.009 / value: 0.020, benefit/cost = 2.2x
($) cost: 21.11 / value: 46.47, average gain = 25.36
-------------------------------
trading put 2175 and call 2375 (hit ratio: 31%)
cost: 0.006 / value: 0.016, benefit/cost = 2.5x
($) cost: 14.53 / value: 36.25, average gain = 21.73


[{'put': 2225.0, 'call': 2325.0, 'quantity': 0},
 {'put': 2200.0, 'call': 2350.0, 'quantity': 1}]

In [9]:
simulation.options_chain

,value_call,value_call_pct,call_ask,value_put,value_put_pct,put_ask
strike,,,,,,
2175.0,114.938104,NaN,0.0535,19.778018,0.008712,0.0035
2200.0,95.049126,NaN,0.0440,24.889040,0.010964,0.0050
2225.0,76.690561,NaN,0.0350,31.530475,0.013889,0.0075
2250.0,60.500794,NaN,0.0205,40.340708,0.017770,0.0110
2275.0,47.006775,0.020706,0.0145,51.846689,NaN,0.0160
2300.0,36.455045,0.016058,0.0100,66.294959,NaN,0.0230
2325.0,28.154131,0.012402,0.0065,82.994045,NaN,NaN
2350.0,21.578549,0.009505,0.0043,101.418463,NaN,NaN
2375.0,16.476585,0.007258,0.0029,121.316499,NaN,NaN
